<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine/DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Data Preparation
#Cleaning: Spellchecks, lemma
#Create embedding matrix, missing embedding imputation

# Build tokenizer and vocab
# create test and train with sequence

# Builf train and evaluate

# Save model for offline scoring on test

In [0]:
# Data preparation

In [2]:
import torch
print("Name of GPU : {}".format(torch.cuda.get_device_name(0)))
print("# of GPU : {}".format(torch.cuda.device_count()))


Name of GPU : Tesla K80
# of GPU : 1


In [0]:
from fastai.text import *
import html
import sklearn
import torch.tensor as T
from sklearn.model_selection import train_test_split

#from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

import pandas as pd
import string
import time
import random
import pandas as pd
import numpy as np
import gc
import re
import torch
from torchtext import data
import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
import zipfile
import gzip

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize
import torch
import torch.nn as nn
import random
import keras
import numpy as np
from keras.layers import Conv1D, Conv2D
from keras import optimizers
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

# cross validation and metrics
from  sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection,linear_model,metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer
from gensim.models.keyedvectors import KeyedVectors

In [0]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/kaggle_toxic_comments/ulmfit"

In [66]:
df = pd.read_csv('/content/drive/My Drive/CCP-ICDsearch/desc-icd9-modeldata/model_data.csv')
#df.head()
# removed duplicated rows
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(16981, 2)
(8705, 2)


In [0]:
#misspell correction
def build_vocab(texts):
    sentences = texts.apply(lambda x: re.split(' |;|,|\*|\n|[(]|[)]',x))
    dfvocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                dfvocab[word] += 1
            except KeyError:
                dfvocab[word] = 1
    return dfvocab
dfvocab = build_vocab(df['desc'])

#len(vocab.keys()) 3630

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
misspells=[]
for k in list(dfvocab.keys()):
    if k in model.vocab:
        continue
    else:
        misspells.append(k)

In [81]:
len(misspells)

743

In [0]:
dfvocab['duodenal']

In [0]:
mispell_dict = {"I'm": "i am","gleekvampire":"gleek vampire","publiclaly":"publicly","rleigion":"religion","easialy":"easily","orthodaox":"orthodox","beelifs":"beliefs","inteliigence":"intelligence","sturcuture":"structure","imposble":"impossible","clarfification":"clarification","temporararily":"temporarily","broughted":"brought","identifiing":"identifying", "noerthern":"northern","rpublic":"republic","immadiately":"immediately","statenment":"statement","yourplaceneeded":"your place needed","thatwasit":"that was it","theclubworkers":"the club workers","megaaward":"mega award","morecurrentevents":"more current events","permanat":"permanent","conndoms":"condoms","sisterproject":"sister project", "editorwelcome":"editor welcome","hindusism":"hinduism","last2":"last 2","emptylineexception":"empty line exception","penisboys":"penis boys","zealousy":"zealous","underpromotion":"under promotion","timeaftertime":"time after time","aeroshuttle":"aero shuttle","constitude":"constitute","emotionallly":"emotionally","circambulation":"circumambulation","flyhead":"fly head","ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am","•":".", "i've": "i have","article's":"article", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have","admins":"admin", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp',"re:":"regarding", "fucksex":"fuck sex","mothjer":"mother","yourselfgo":"yourself go",'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x): 
    #x = re.sub(r"\W+", " ", x.lower())
    x = bytes(x, encoding="utf-8")
    x = x.replace(b"\n", b" ")
    x = x.replace(b"\t", b" ")
    x = x.replace(b"\b", b" ")
    x = x.replace(b"\r", b" ")
    x = re.sub(b"\s+", b" ", x)
    x = str(x, 'utf-8')
    x = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", x).strip()
    x = re.sub(r'(.)\1{2,}', r'\1\1', x)
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def replace_n_punc(x):
  return re.sub(r'[\?\.\!\,\=]+(?=[\?\.\!\,\=])', '', x)
    
  
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [0]:
train_df = pd.read_csv("/content/drive/My Drive/kaggle_toxic_comments/train.csv")
test_df = pd.read_csv("/content/drive/My Drive/kaggle_toxic_comments/test.csv")

train_X = train_df
test_X = test_df
# lower
train_X["comment_text"] = train_df["comment_text"].apply(lambda x: x.lower())
test_X["comment_text"] = test_df["comment_text"].apply(lambda x: x.lower())

# Clean speelings
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: replace_typical_misspell(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: replace_typical_misspell(x))
   
# Replcae multiple punct by single
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: replace_n_punc(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: replace_n_punc(x))    

# Clean numbers
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: clean_numbers(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: clean_numbers(x))

# Clean the text
train_X["comment_text"] = train_X["comment_text"].progress_apply(lambda x: clean_text(x))
test_X["comment_text"] = test_X["comment_text"].apply(lambda x: clean_text(x))

## fill up the missing values
train_X["comment_text"].fillna("_##_", inplace=True)
test_X["comment_text"].fillna("_##_", inplace=True)